In [3]:
words = open('names.txt', 'r').read().splitlines()

Creating bigrams - two letter objects capturing what letter is likely to follow another
adding a start and an end characters to capture what letter to start with and with which letter the word is likely to end the word

In [4]:
b={}
for w in words[:1]:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs,chs[:1]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) +1 # counting the number that a bigram occurs


storing the bigram information in a 2D array (pytorch tensors) where the rows are the first character and the columns are the second character

In [5]:
import torch

In [6]:
N = torch.zeros((27,27), dtype=torch.int32)

chars = sorted(list(set(''.join(words ))))
stoi = {s:i+1 for i,s in enumerate(chars)} #mapping of a character to an index
stoi['.'] = 0 # replacing special end and start characters with a single . representing the space between words
itos = {i:s for s,i in stoi.items()}
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2] # index of the character - defining position in the array
        N[ix1,ix2] += 1

In [ ]:
P = N.float() #convert to float cause we want to devide
P /= P.sum(1, keepdim=True) # normalize occurences , in-place operation has potential to be faster - doesnt create more memory under-the-hood


In [8]:
g = torch.Generator().manual_seed(2147484647)
for i in range(20):
    ix = 0
    out=[]
    while True:
        p=P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item() #obtianing normal distribuition from probabilities 
        out.append(itos[ix])
        if ix ==0:
            break
    print(''.join(out))

nieran.
cele.
brolyah.
ananei.
egin.
baijeriesetengh.
bebynnojarzakia.
h.
n.
er.
xlah.
assaiavees.
be.
drikyly.
en.
d.
an.
ja.
ma.
am.


when printing the probabilities of bigrams in real words we want the total probability according to a well trained model to be close to 1 (maximize the likelyhood). we can use log(likelyhood) for convenience - total prob = product of all probabilities which will be a tiny number for many words - log_likelyhood is a more natural number. 
negative log likelyhood - good loss function minimum is 0. use the average -.

In [9]:
log_likelyhood = 0.0
n=0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2] 
        prob = P[ix1,ix2]
        log_likelyhood += torch.log(prob)
        n+=1
nll = -log_likelyhood
print(f'{nll/n}')


2.454094171524048
